# Payout Calculator

In [2]:
import pandas as pd
import itertools
import selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## Screpe

In [194]:
PATH = "/Users/andrewjfarrell1/Documents/Horse/PayoutCalculator/chromedriver"
driver = webdriver.Chrome(PATH)

driver.get("https://tvg.com")
# print(driver.title)

time.sleep(1)

close_popup = driver.find_element("id","closeQubitModal")
close_popup.click()

link = driver.find_element("link text","Bet Saratoga")
link.click()

time.sleep(1)

# dollars =driver.find_element("xpath","//div[@class='game_purchase_price price']").get_attribute("data-price-final")
# # print(value)

# dollars =driver.find_element("xpath","//div[@class='result-runners-table pools-table']")
# print(dollars.text)

dollars = driver.find_element("css selector","div.result-runners-table.pools-table")
amounts = dollars.find_elements("tag name","runner-amounts")
leest = []
for amount in amounts:
    leest.append(amount.text)
    
    

# from io import StringIO
# WPS_df = pd.read_csv(StringIO(dollars.text), sep=r"\s\s+",engine='python')

# WPS_df

# try:
#     dollars = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CSS_SELECTOR, "result-runners-table pools-table"))
#     )
#     print(dollars.text)
    
# except:
#     driver.quit()


driver.quit()

new_leest = []
for elem in leest:
    elem = elem.replace(",","")
    elem = elem.replace("$","")
    new_leest.append(elem)
    
newer_leest = []
for elem in new_leest:
    elem = elem.replace(" ",",")
    newer_leest.append(elem)

final_leest = []
for elem in newer_leest:
    elem = elem.split(",")
    final_leest.append(elem)
boop = {elem[0]:elem[1:] for elem in final_leest}
data = pd.DataFrame.from_dict(boop,orient='index')
data.columns = ["Win Pool", "Place Pool", "Show Pool"]
data.index.name = "Horse Number"
data
data.reset_index(inplace=True)
data = data.apply(pd.to_numeric)

<ipython-input-194-ef2a018559f6>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [195]:
# data = pd.read_csv("Race10_Aug14.csv")
# data

In [196]:
n = len(data.index)
horses = list(range(1,n+1))
horses

[1, 2, 3, 4, 5, 6]

In [197]:
place_perms = list(itertools.permutations(horses,2))
show_perms = list(itertools.permutations(horses,3))

## Win Payouts

In [198]:
win_df = pd.DataFrame(horses)
win_df.columns = ["1st Place Finisher"]
win_df

,1st Place Finisher
0,1
1,2
2,3
3,4
4,5
5,6


In [199]:
win_dict = dict(zip(data["Horse Number"],data["Win Pool"]))
win_dict

{1: 9389, 2: 4640, 3: 7353, 4: 6713, 5: 17829, 6: 0}

In [200]:
win_df["1st Place Finisher Win Amount"] = win_df["1st Place Finisher"].map(win_dict)
# win_df["odds"] = race6["Odds"]
win_df

,1st Place Finisher,1st Place Finisher Win Amount
0,1,9389
1,2,4640
2,3,7353
3,4,6713
4,5,17829
5,6,0


In [201]:
win_df["Win Payout"] = ((data["Win Pool"].sum()*0.84-win_df["1st Place Finisher Win Amount"]))/win_df["1st Place Finisher Win Amount"]
win_df

,1st Place Finisher,1st Place Finisher Win Amount,Win Payout
0,1,9389,3.108655
1,2,4640,7.313828
2,3,7353,4.246316
3,4,6713,4.746486
4,5,17829,1.163675
5,6,0,inf


## Place Payouts

In [202]:
place_df = pd.DataFrame(place_perms)
place_df.columns = ["1st Place Finisher", "2nd Place Finisher"]
place_df

,1st Place Finisher,2nd Place Finisher
0,1,2
1,1,3
2,1,4
3,1,5
4,1,6
5,2,1
6,2,3
7,2,4
8,2,5
9,2,6


In [203]:
place_dict = dict(zip(data["Horse Number"],data["Place Pool"]))
place_dict

{1: 4701, 2: 1842, 3: 3698, 4: 2324, 5: 4087, 6: 0}

In [204]:
place_df["1st Place Finisher Place Amount"] = place_df["1st Place Finisher"].map(place_dict)
place_df["2nd Place Finisher Place Amount"] = place_df["2nd Place Finisher"].map(place_dict)
place_df

,1st Place Finisher,2nd Place Finisher,1st Place Finisher Place Amount,2nd Place Finisher Place Amount
0,1,2,4701,1842
1,1,3,4701,3698
2,1,4,4701,2324
3,1,5,4701,4087
4,1,6,4701,0
5,2,1,1842,4701
6,2,3,1842,3698
7,2,4,1842,2324
8,2,5,1842,4087
9,2,6,1842,0


In [205]:
place_df["Sum of 1st & 2nd Place Pools"] = place_df["1st Place Finisher Place Amount"]+place_df["2nd Place Finisher Place Amount"]
place_df["Place Payout 1"] = ((data["Place Pool"].sum()*0.84-place_df["Sum of 1st & 2nd Place Pools"])/2)/place_df["1st Place Finisher Place Amount"]
place_df["Place Payout 2"] = ((data["Place Pool"].sum()*0.84-place_df["Sum of 1st & 2nd Place Pools"])/2)/place_df["2nd Place Finisher Place Amount"]
place_df

,1st Place Finisher,2nd Place Finisher,1st Place Finisher Place Amount,2nd Place Finisher Place Amount,Sum of 1st & 2nd Place Pools,Place Payout 1,Place Payout 2
0,1,2,4701,1842,6543,0.791819,2.020814
1,1,3,4701,3698,8399,0.594414,0.755635
2,1,4,4701,2324,7025,0.740553,1.497995
3,1,5,4701,4087,8788,0.553040,0.636124
4,1,6,4701,0,4701,0.987735,inf
5,2,1,1842,4701,6543,2.020814,0.791819
6,2,3,1842,3698,5540,2.293073,1.142196
7,2,4,1842,2324,4166,2.666037,2.113098
8,2,5,1842,4087,5929,2.187481,0.985892
9,2,6,1842,0,1842,3.296873,inf


In [206]:
Max = pd.DataFrame(place_df.groupby('1st Place Finisher').max()['Place Payout 1'])
Min = pd.DataFrame(place_df.groupby('1st Place Finisher').min()['Place Payout 1'])
Place_Payouts = pd.merge(Max,Min, on ='1st Place Finisher')
Place_Payouts = Place_Payouts.reset_index()
Place_Payouts.columns = ['Horse','Max Place Payout', 'Min Place Payout']
Place_Payouts

,Horse,Max Place Payout,Min Place Payout
0,1,0.987735,0.553040
1,2,3.296873,2.020814
2,3,1.391249,0.755635
3,4,2.509398,1.497995
4,5,1.211241,0.636124
5,6,inf,inf


In [207]:
winninghorses = list(map(int, input("Enter 1st & 2nd Place Horses (separated by comma): ").split(',')))

Enter 1st & 2nd Place Horses (separated by comma): 1,2


In [208]:
def HowMuchDoIWin(winninghorses):
    a = winninghorses[0]
    b = winninghorses[1]
    return place_df.loc[(place_df['1st Place Finisher'] == a ) & (place_df['2nd Place Finisher'] == b)]
win = HowMuchDoIWin(winninghorses)
win

,1st Place Finisher,2nd Place Finisher,1st Place Finisher Place Amount,2nd Place Finisher Place Amount,Sum of 1st & 2nd Place Pools,Place Payout 1,Place Payout 2
0,1,2,4701,1842,6543,0.791819,2.020814


## Show Payouts

In [209]:
show_df = pd.DataFrame(show_perms)
show_df.columns = ["1st Place Finisher", "2nd Place Finisher","3rd Place Finisher"]
show_df

,1st Place Finisher,2nd Place Finisher,3rd Place Finisher
0,1,2,3
1,1,2,4
2,1,2,5
3,1,2,6
4,1,3,2
...,...,...,...
115,6,4,5
116,6,5,1
117,6,5,2
118,6,5,3


In [210]:
show_dict = dict(zip(data["Horse Number"],data["Show Pool"]))
show_dict

{1: 3445, 2: 1254, 3: 2194, 4: 1437, 5: 2498, 6: 0}

In [211]:
show_df["1st Place Finisher Show Amount"] = show_df["1st Place Finisher"].map(show_dict)
show_df["2nd Place Finisher Show Amount"] = show_df["2nd Place Finisher"].map(show_dict)
show_df["3rd Place Finisher Show Amount"] = show_df["3rd Place Finisher"].map(show_dict)
show_df

,1st Place Finisher,2nd Place Finisher,3rd Place Finisher,1st Place Finisher Show Amount,2nd Place Finisher Show Amount,3rd Place Finisher Show Amount
0,1,2,3,3445,1254,2194
1,1,2,4,3445,1254,1437
2,1,2,5,3445,1254,2498
3,1,2,6,3445,1254,0
4,1,3,2,3445,2194,1254
...,...,...,...,...,...,...
115,6,4,5,0,1437,2498
116,6,5,1,0,2498,3445
117,6,5,2,0,2498,1254
118,6,5,3,0,2498,2194


In [212]:
show_df["Sum of 1st, 2nd, and 3rd Show Pools"] = show_df["1st Place Finisher Show Amount"]+show_df["2nd Place Finisher Show Amount"]+show_df["3rd Place Finisher Show Amount"]
show_df["Show Payout 1"] = ((data["Show Pool"].sum()*0.84-show_df["Sum of 1st, 2nd, and 3rd Show Pools"])/3)/show_df["1st Place Finisher Show Amount"]
show_df["Show Payout 2"] = ((data["Show Pool"].sum()*0.84-show_df["Sum of 1st, 2nd, and 3rd Show Pools"])/3)/show_df["2nd Place Finisher Show Amount"]
show_df["Show Payout 3"] = ((data["Show Pool"].sum()*0.84-show_df["Sum of 1st, 2nd, and 3rd Show Pools"])/3)/show_df["3rd Place Finisher Show Amount"]
show_df

,1st Place Finisher,2nd Place Finisher,3rd Place Finisher,1st Place Finisher Show Amount,2nd Place Finisher Show Amount,3rd Place Finisher Show Amount,"Sum of 1st, 2nd, and 3rd Show Pools",Show Payout 1,Show Payout 2,Show Payout 3
0,1,2,3,3445,1254,2194,6893,0.213113,0.585465,0.334628
1,1,2,4,3445,1254,1437,6136,0.286359,0.786688,0.686504
2,1,2,5,3445,1254,2498,7197,0.183698,0.504657,0.253339
3,1,2,6,3445,1254,0,4699,0.425401,1.168666,inf
4,1,3,2,3445,2194,1254,6893,0.213113,0.334628,0.585465
...,...,...,...,...,...,...,...,...,...,...
115,6,4,5,0,1437,2498,3935,inf,1.197059,0.688620
116,6,5,1,0,2498,3445,5943,inf,0.420673,0.305033
117,6,5,2,0,2498,1254,3752,inf,0.713040,1.420393
118,6,5,3,0,2498,2194,4692,inf,0.587606,0.669025


In [213]:
Max_show = pd.DataFrame(show_df.groupby('1st Place Finisher').max()['Show Payout 1'])
Min_show = pd.DataFrame(show_df.groupby('1st Place Finisher').min()['Show Payout 1'])
Show_Payouts = pd.merge(Max_show,Min_show, on ='1st Place Finisher')
Show_Payouts = Show_Payouts.reset_index()
Show_Payouts.columns = ['Horse','Max Show Payout', 'Min Show Payout']
Show_Payouts

,Horse,Max Show Payout,Min Show Payout
0,1,0.425401,0.092745
1,2,1.702424,0.504657
2,3,0.858025,0.145627
3,4,1.485623,0.397940
4,5,0.713040,0.127905
5,6,inf,inf


In [214]:
winninghorses_show = list(map(int, input("Enter first 3 finishing horses (separated by comma): ").split(',')))

Enter first 3 finishing horses (separated by comma): 1,2,3


In [215]:
def HowMuchDoIWin_Show(winninghorses_show):
    a = winninghorses_show[0]
    b = winninghorses_show[1]
    c = winninghorses_show[2]
    return show_df.loc[(show_df['1st Place Finisher'] == a ) & (show_df['2nd Place Finisher'] == b) & (show_df['3rd Place Finisher'] == c)]
show = HowMuchDoIWin_Show(winninghorses_show)
show

,1st Place Finisher,2nd Place Finisher,3rd Place Finisher,1st Place Finisher Show Amount,2nd Place Finisher Show Amount,3rd Place Finisher Show Amount,"Sum of 1st, 2nd, and 3rd Show Pools",Show Payout 1,Show Payout 2,Show Payout 3
0,1,2,3,3445,1254,2194,6893,0.213113,0.585465,0.334628


In [216]:
wombo_combo = pd.concat([Place_Payouts,Show_Payouts,win_df],axis=1)
wombo_combo

,Horse,Max Place Payout,Min Place Payout,Horse,Max Show Payout,Min Show Payout,1st Place Finisher,1st Place Finisher Win Amount,Win Payout
0,1,0.987735,0.553040,1,0.425401,0.092745,1,9389,3.108655
1,2,3.296873,2.020814,2,1.702424,0.504657,2,4640,7.313828
2,3,1.391249,0.755635,3,0.858025,0.145627,3,7353,4.246316
3,4,2.509398,1.497995,4,1.485623,0.397940,4,6713,4.746486
4,5,1.211241,0.636124,5,0.713040,0.127905,5,17829,1.163675
5,6,inf,inf,6,inf,inf,6,0,inf


In [217]:
wombo_combo = wombo_combo.loc[:,~wombo_combo.columns.duplicated()]
wombo_combo = wombo_combo.drop(columns=['1st Place Finisher','1st Place Finisher Win Amount'],axis=1)

In [218]:
wombo_combo['Place_range'] = wombo_combo['Max Place Payout'] - wombo_combo['Min Place Payout']
wombo_combo['Show_range'] = wombo_combo['Max Show Payout'] - wombo_combo['Min Show Payout']
wombo_combo

,Horse,Max Place Payout,Min Place Payout,Max Show Payout,Min Show Payout,Win Payout,Place_range,Show_range
0,1,0.987735,0.553040,0.425401,0.092745,3.108655,0.434695,0.332656
1,2,3.296873,2.020814,1.702424,0.504657,7.313828,1.276059,1.197767
2,3,1.391249,0.755635,0.858025,0.145627,4.246316,0.635614,0.712397
3,4,2.509398,1.497995,1.485623,0.397940,4.746486,1.011403,1.087683
4,5,1.211241,0.636124,0.713040,0.127905,1.163675,0.575116,0.585135
5,6,inf,inf,inf,inf,inf,NaN,NaN


In [219]:
wombo_combo['Win Over MaxP'] = wombo_combo['Win Payout']/wombo_combo['Max Place Payout']
wombo_combo['Win Over MinP'] = wombo_combo['Win Payout']/wombo_combo['Min Place Payout']
wombo_combo['Win Over MaxS'] = wombo_combo['Win Payout']/wombo_combo['Max Show Payout']
wombo_combo['Win Over MinS'] = wombo_combo['Win Payout']/wombo_combo['Min Show Payout']
wombo_combo

,Horse,Max Place Payout,Min Place Payout,Max Show Payout,Min Show Payout,Win Payout,Place_range,Show_range,Win Over MaxP,Win Over MinP,Win Over MaxS,Win Over MinS
0,1,0.987735,0.553040,0.425401,0.092745,3.108655,0.434695,0.332656,3.147257,5.621033,7.307586,33.518286
1,2,3.296873,2.020814,1.702424,0.504657,7.313828,1.276059,1.197767,2.218414,3.619248,4.296125,14.492668
2,3,1.391249,0.755635,0.858025,0.145627,4.246316,0.635614,0.712397,3.052160,5.619529,4.948942,29.158756
3,4,2.509398,1.497995,1.485623,0.397940,4.746486,1.011403,1.087683,1.891484,3.168560,3.194947,11.927638
4,5,1.211241,0.636124,0.713040,0.127905,1.163675,0.575116,0.585135,0.960730,1.829320,1.631992,9.097963
5,6,inf,inf,inf,inf,inf,NaN,NaN,NaN,NaN,NaN,NaN


## Screpe Example

In [220]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# PATH = "/Users/andrewjfarrell1/Documents/Horse/PayoutCalculator/chromedriver"
# driver = webdriver.Chrome(PATH)

# driver.get("https://techwithtim.net")
# print(driver.title)

# search = driver.find_element("name","s")
# search.send_keys("test") #test is what you are searching in search bar
# search.send_keys(Keys.RETURN)
# # print(driver.page_source)

# # time.sleep(10)

# try:
#     main = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, "main"))
#     )
#     articles = main.find_elements("tag name","article")
#     for article in articles:
#         summary = article.find_element("class name","entry-summary")
#         print(summary.text)
# except:
#     driver.quit()
    
# # main = driver.find_element("id","main")

# driver.back()

# link = driver.find_element("link text","Python Programming")
# link.click()

# try:
#     element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.LINK_TEXT, "Beginner Python Tutorials"))
#     )
#     element.click() 
#     second_element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, "sow-button-19310003"))
#     )
#     second_element.click()
    
# except:
#     driver.quit()

# time.sleep(5)
# driver.quit()